In [9]:
# For every routine
# Sample times per activity
# Check for overlaps and get actions in order
# Create single script with times
# Execute to get timestamped graphs ***

In [10]:
%load_ext autoreload
%autoreload 2

import json
import os
import shutil
import numpy as np
from random import random

from evolving_graph.scripts import Script, parse_script_line
import evolving_graph.utils as utils
from evolving_graph.scripts import read_script, read_script_from_string, read_script_from_list_string, ScriptParseException, Action
from evolving_graph.execution import ScriptExecutor
from evolving_graph.environment import EnvironmentGraph

from GraphReader import GraphReader, init_graph_file, scene_num
from ProgramExecutor import read_program

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def time_internal(mins, hrs, day):
    return int(round(((((day)*24)+hrs)*60+mins)/DT))

def day_num(day_of_week):
    return {'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}[day_of_week]

In [ ]:
init_graph = GraphReader(graph_file=init_graph_file)
print(f'Using scene {int(scene_num)-1}, i.e. \'TestScene{scene_num}\'')

In [ ]:
DT = 1   # minutes
NUM_REPEAT_ROUTINES = 5
WEEKEND_DAYS = [day_num(day) for day in ['Saturday','Sunday']]
INTERLEAVING = False
START_TIME = (5,50)
END_TIME = (24,00)

DATASET_DIR = 'sourcedRoutines/trial'
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)
os.makedirs(DATASET_DIR)

In [ ]:
class Activity():
    def __init__(self, name, time, script_file=None):
        self.name = name
        start_time = time[0][0]*60 + time[0][1]
        end_time = time[1][0]*60 + time[1][1]
        directory = os.path.join('sourcedScripts', name)
        if script_file is None:
            script_file = np.random.choice(os.listdir(directory))
            headers, self.scripts = read_program(os.path.join(directory,script_file), init_graph.node_map)
        def sample_duration(header):
            durations = (header).split('-')
            assert len(durations)==2, f"Invalid comment {header} in {name}->{script_file}"
            duration_min = int(durations[0].strip())
            duration_max = int(durations[1].strip())
            duration_sampled = random() * (duration_max-duration_min) + duration_min
            return duration_sampled
        self.durations = [sample_duration(header) for header in headers]
        def valid_times(times):
            for time, next_time, duration in zip(times[:-1], times[1:], self.durations[:-1]):
                if next_time-time < duration:
                    return False
            return True
        valid = False
        while not valid:
            times = (np.random.rand(len(headers)) * (end_time-start_time) + start_time).round().astype(int)
            times.sort()
            valid = valid_times(times)
        self.times = times


class Schedule():
    def __init__(self, sourced_schedule):


In [12]:
schedule_file = 'sourcedSchedules/example.json'
with open(schedule_file) as f:
    schedule = json.load(f)

{'getting_out_of_bed': [[6, 0], [9, 0]],
 'brushing_teeth': [[6, 0], [9, 0]],
 'showering': [[7, 0], [9, 0]],
 'breakfast': [[7, 0], [9, 0]],
 'getting_dressed': [[9, 0], [10, 0]],
 'leave_home': [[10, 0], [12, 0]],
 'coming_back_home': [[5, 0], [6, 0]],
 'watching_tv': [[5, 0], [6, 0]],
 'dinner': [[6, 0], [8, 0]],
 'take_out_trash': [[8, 0], [9, 0]],
 'taking_medication': [[8, 30], [9, 30]],
 'sleeping': [[10, 0], [11, 0]]}